In [1]:
# -*- coding: utf-8 -*-
import os
import pickle
from typing import List

import pandas
import numpy as np
from tqdm import tqdm

In [2]:
orig_name = 'dataset II-3/SITE-Seq_offTarget_wholeDataset.csv'
csv_name = 'II-3'

In [3]:
'''
读 csv
'''
assert os.path.exists('../../data')
dataset = pandas.read_csv('../../data/Dataset II (mismatch-only)/%s' % orig_name, delimiter=',')
dataset

,Unnamed: 0,direction,location,off_seq,on_seq,reads,on_off_sp
0,0,-,chr1:13386,GGAGGCTCTAGGGAAAGGAAAAG,GGGGCCACTAGGGACAGGATNGG,0.0,GGAGGCTCTAGGGAAAGGAAAAGGGGGCCACTAGGGACAGGATNGG
1,1,+,chr1:158621,TGGGAAACTAGGGACAGTACTTG,GGGGCCACTAGGGACAGGATNGG,0.0,TGGGAAACTAGGGACAGTACTTGGGGGCCACTAGGGACAGGATNGG
2,2,-,chr1:427716,GGGGGCACAGGAGACAGGCCTGG,GGGGCCACTAGGGACAGGATNGG,0.0,GGGGGCACAGGAGACAGGCCTGGGGGGCCACTAGGGACAGGATNGG
3,3,+,chr1:556867,GGGGGCACAGGAGACAGGCCTGG,GGGGCCACTAGGGACAGGATNGG,0.0,GGGGGCACAGGAGACAGGCCTGGGGGGCCACTAGGGACAGGATNGG
4,4,+,chr1:812137,GGGGGCTTCATGGACAGGAGTGG,GGGGCCACTAGGGACAGGATNGG,0.0,GGGGGCTTCATGGACAGGAGTGGGGGGCCACTAGGGACAGGATNGG
...,...,...,...,...,...,...,...
217728,217728,+,chr5:2282405-2282427,TGATGGCCAAGGACAGATAGAGG,GGTGGACAAGCGGCAGATAGNGG,11.0,TGATGGCCAAGGACAGATAGAGGGGTGGACAAGCGGCAGATAGNGG
217729,217729,+,chr4:87292680-87292702,GGGGGACAAGCGGCAGAGGGGGA,GGTGGACAAGCGGCAGATAGNGG,11.0,GGGGGACAAGCGGCAGAGGGGGAGGTGGACAAGCGGCAGATAGNGG
217730,217730,-,chr14:49882638-49882660,GAATGAACAAGGACAGATAGGAG,GGTGGACAAGCGGCAGATAGNGG,11.0,GAATGAACAAGGACAGATAGGAGGGTGGACAAGCGGCAGATAGNGG
217731,217731,-,chr1:184926753-184926775,GGAGGTAGACAGACAGATAGGGG,GGTGGACAAGCGGCAGATAGNGG,11.0,GGAGGTAGACAGACAGATAGGGGGGTGGACAAGCGGCAGATAGNGG


In [4]:
on = dataset['on_seq']
off = dataset['off_seq']
reads = dataset['reads']
reads = np.array(reads)
val = np.zeros(len(reads))
val[reads > 0] = 1
on,off,val

(0         GGGGCCACTAGGGACAGGATNGG
 1         GGGGCCACTAGGGACAGGATNGG
 2         GGGGCCACTAGGGACAGGATNGG
 3         GGGGCCACTAGGGACAGGATNGG
 4         GGGGCCACTAGGGACAGGATNGG
                    ...           
 217728    GGTGGACAAGCGGCAGATAGNGG
 217729    GGTGGACAAGCGGCAGATAGNGG
 217730    GGTGGACAAGCGGCAGATAGNGG
 217731    GGTGGACAAGCGGCAGATAGNGG
 217732    GGTGGACAAGCGGCAGATAGNGG
 Name: on_seq, Length: 217733, dtype: object,
 0         GGAGGCTCTAGGGAAAGGAAAAG
 1         TGGGAAACTAGGGACAGTACTTG
 2         GGGGGCACAGGAGACAGGCCTGG
 3         GGGGGCACAGGAGACAGGCCTGG
 4         GGGGGCTTCATGGACAGGAGTGG
                    ...           
 217728    TGATGGCCAAGGACAGATAGAGG
 217729    GGGGGACAAGCGGCAGAGGGGGA
 217730    GAATGAACAAGGACAGATAGGAG
 217731    GGAGGTAGACAGACAGATAGGGG
 217732    ATGGATGGATAGGTAGATAGAGG
 Name: off_seq, Length: 217733, dtype: object,
 array([0., 0., 0., ..., 1., 1., 1.]))

In [5]:
tlen =24
ons = []
offs = []
for i in range(len(on)):
    ons.append('-'*(tlen-len(on[i]))+on[i])
for i in range(len(off)):
    offs.append('-'*(tlen-len(off[i]))+off[i])

In [6]:
max([len(s) for s in ons])

24

In [7]:
onss = []
for i in tqdm(range(len(ons))):
    seq = []
    for j in range(tlen):
        if ons[i][j] == 'N':
            seq.append(offs[i][j])
        else:
            seq.append(ons[i][j])
    onss.append(''.join(seq))
onss[0:10]

100%|██████████| 217733/217733 [00:01<00:00, 185425.88it/s]


['-GGGGCCACTAGGGACAGGATAGG',
 '-GGGGCCACTAGGGACAGGATTGG',
 '-GGGGCCACTAGGGACAGGATTGG',
 '-GGGGCCACTAGGGACAGGATTGG',
 '-GGGGCCACTAGGGACAGGATTGG',
 '-GGGGCCACTAGGGACAGGATGGG',
 '-GGGGCCACTAGGGACAGGATTGG',
 '-GGGGCCACTAGGGACAGGATGGG',
 '-GGGGCCACTAGGGACAGGATAGG',
 '-GGGGCCACTAGGGACAGGATGGG']

In [8]:
offss = []
for i in tqdm(range(len(offs))):
    seq = []
    for j in range(tlen):
        if offs[i][j] == 'N':
            seq.append(onss[i][j])
        else:
            seq.append(offs[i][j])
    offss.append(''.join(seq))
offss[0:10]

100%|██████████| 217733/217733 [00:01<00:00, 183385.77it/s]


['-GGAGGCTCTAGGGAAAGGAAAAG',
 '-TGGGAAACTAGGGACAGTACTTG',
 '-GGGGGCACAGGAGACAGGCCTGG',
 '-GGGGGCACAGGAGACAGGCCTGG',
 '-GGGGGCTTCATGGACAGGAGTGG',
 '-GAGGCCGCGCGCGATAGGATGGG',
 '-TGGGAAACTAGGGACAGTACTTG',
 '-GGGGCCACAGGATGCAGGGTGGG',
 '-GGCGGCAGTGAGGACAGGAGAGG',
 '-GGGACCACTGACCACAGGCTGGG']

In [9]:
df = pandas.DataFrame(data={
    'on': onss,
    'off': offss,
    'val': val,
})
df.to_csv('../../data/%s.csv' % csv_name, index=False)